In [59]:
import requests
base_url = 'http://localhost:7999'

In [60]:
class Session(requests.Session):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.headers.update({'Content-Type': 'application/json'})
    
    def get(self, url, *args, **kwargs):
        return super().get(base_url + url, *args, **kwargs)
    
    def post(self, url, *args, **kwargs):
        return super().post(base_url + url, *args, **kwargs)
    
    def patch(self, url, *args, **kwargs):
        return super().patch(base_url + url, *args, **kwargs)
    
    def delete(self, url, *args, **kwargs):
        return super().delete(base_url + url, *args, **kwargs)

client = Session()
resp = client.get("/ping")
resp.json()

{'data': 'pong!'}

In [54]:
# import os
# import sys

# backend_path = os.path.abspath(os.path.join(os.getcwd(), '../../backend'))
# sys.path.append(backend_path)
# from app.internal.firebase import db

In [53]:
query = "Nip IND ETF Junior BeES".upper()
print(query)


docs = await db.collection("tickers").where("name", ">=", query).where("name", "<=", query + '\uf8ff').get()
# doc = await db.collection("tickers").where("ticker", "<=", query).where("ticker", ">=", query + "~").get()

if docs:
    print(docs[0].to_dict())

print(docs)

NIP IND ETF JUNIOR BEES
{'created_at': '2024-05-05T12:20:40.104545Z', 'exchange': 'NSE', 'lot_size': 1, 'upstox_instrument_key': 'NSE_EQ|INF732E01045', 'exchange_token': 10939, 'updated_at': '2024-05-05T12:20:40.104547Z', 'ticker': 'JUNIORBEES', 'instrument_type': 'EQUITY', 'name': 'NIP IND ETF JUNIOR BEES'}


/var/folders/lq/280gf8ld14v161g6ngpdjjnc0000gn/T/ipykernel_8262/3214706254.py:5: UserWarning: Detected filter using positional arguments. Prefer using the 'filter' keyword argument instead.
  docs = await db.collection("tickers").where("name", ">=", query).where("name", "<=", query + '\uf8ff').get()


In [61]:
from datetime import datetime
date = datetime.now()
print(date.strftime("%Y-%m-%d"))

2024-05-05


In [62]:
def format_date_path(date: datetime) -> str:
    return date.strftime("%Y-%m-%d")

In [5]:
from datetime import datetime
from typing import Dict, Optional, Annotated
from pydantic import BaseModel, Field, validator, StringConstraints

def format_date_path(date: datetime) -> str:
    try:
        return date.strftime("%Y-%m-%d")
    except:
        raise ValueError(f"Invalid date format {date}")
        
def validate_date_path(date: str):
    try:
        datetime.strptime(date, "%Y-%m-%d")
    except:
        raise ValueError(f"Invalid date format {date}")

class CandleBase(BaseModel):
    open: float
    high: float
    low: float
    close: float
    volume: int
    open_interest: Optional[int] = None

class MonthlyCandleBase(BaseModel):
    month: Annotated[str, StringConstraints(pattern=r"^(0[1-9]|1[0-2])$")]
    year: Annotated[str, StringConstraints(pattern=r"^(19\d{2}|20\d{2})$")]
    daily: Dict[str, CandleBase] = Field(default_factory=dict)
    monthly: Optional[CandleBase] = None

    @validator("daily", pre=True, allow_reuse=True)
    def validate_daily(cls, v):
        for key in v:
            validate_date_path(key)
        return v

/var/folders/lq/280gf8ld14v161g6ngpdjjnc0000gn/T/ipykernel_52500/2471255407.py:31: PydanticDeprecatedSince20: Pydantic V1 style `@validator` validators are deprecated. You should migrate to Pydantic V2 style `@field_validator` validators, see the migration guide for more details. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.6/migration/
  @validator("daily", pre=True, allow_reuse=True)


In [8]:
date = "2021-01-01"


payload = {
    "month": "01",
    "year": "2021",
    "daily": {
        "2021-01-01": {
            "open": 100,
            "high": 110,
            "low": 90,
            "close": 100,
            "volume": 1000
        },
        "2022-01-002": {
        "open": 100,
        "high": 120,
        "low": 80,
        "close": 90,
        "volume": 800
        }
    },
    "monthly": {
        "open": 100,
        "high": 110,
        "low": 90,
        "close": 100,
        "volume": 1000
    }
}
candle = MonthlyCandleBase(**payload)
candle

ValidationError: 1 validation error for MonthlyCandleBase
daily
  Value error, Invalid date format 2022-01-002 [type=value_error, input_value={'2021-01-01': {'open': 1...se': 90, 'volume': 800}}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.6/v/value_error